In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import tarfile
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import logging
from google.colab import drive

# Монтируем Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import logging

# Настройки логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Основные параметры и настройки
base_urls = [
    "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/00/",
    "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/01/",
    "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/02/",
    "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/03/",
    "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/04/",
    "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/05/",
    "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/06/",
    "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/07/",
    "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/08/",
    "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/09/",
]

# Директории для временных файлов в сессии
save_directory = "/content/pmc_files"
extracted_directory = "/content/extracted_xml_files"

# Директория для готовых CSV-файлов на Google Диске
csv_output_dir = "/content/drive/MyDrive/Colab Notebooks/pmc_articles/raw_population_data/csv_files"  # Директория для сохранения CSV-файлов

max_files = 20000
num_threads = 10

# Создание директорий, если они не существуют
os.makedirs(save_directory, exist_ok=True)
os.makedirs(extracted_directory, exist_ok=True)
os.makedirs(csv_output_dir, exist_ok=True)

# Логирование для подтверждения создания директорий
logging.info(f"Директория для сохранения файлов: {save_directory}")
logging.info(f"Директория для извлеченных XML файлов: {extracted_directory}")
logging.info(f"Директория для готовых CSV файлов: {csv_output_dir}")

In [ ]:
# Функция для получения списка файлов из директории на сервере по указанному URL
def get_files_from_directory(directory_url):
    response = requests.get(directory_url)
    soup = BeautifulSoup(response.text, "html.parser")
    return [link.get("href") for link in soup.find_all("a") if link.get("href").endswith(".tar.gz")]

# Функция для загрузки одного файла по указанному URL
def download_file(file_url, file_number):
    try:
        logging.info(f"Начинается загрузка файла #{file_number}: {file_url}")
        response = requests.get(file_url, stream=True)
        response.raise_for_status()
        filename = os.path.join(save_directory, os.path.basename(file_url))
        with open(filename, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        logging.info(f"Загружен: {filename}")
    except requests.exceptions.RequestException as e:
        logging.error(f"Ошибка загрузки {file_url}: {e}")

# Функция для загрузки всех файлов из поддиректорий на сервере с использованием многопоточности
def download_all_files(base_url, max_files):
    all_files = []
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, "html.parser")
    subdirectories = [link.get("href") for link in soup.find_all("a") if link.get("href").endswith("/")]

    for subdir in subdirectories:
        subdir_url = f"{base_url}{subdir}"
        files = get_files_from_directory(subdir_url)
        all_files.extend([f"{subdir_url}{file}" for file in files])
        if len(all_files) >= max_files:
            break

    total_files = len(all_files[:max_files])
    logging.info(f"Всего файлов для загрузки: {total_files} из {base_url}")

    # Загрузка файлов с использованием потоков
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        list(tqdm(executor.map(lambda x: download_file(x[1], x[0]), enumerate(all_files[:max_files])), total=total_files, desc="Загрузка", unit="file", ncols=100))

# Функция для распаковки файлов tar.gz
def extract_xml_files(tar_file):
    try:
        with tarfile.open(tar_file, "r:gz") as tar:
            for member in tar.getmembers():
                if member.name.endswith('.nxml') or member.name.endswith('.xml'):
                    member.name = os.path.basename(member.name)
                    tar.extract(member, extracted_directory)
                    logging.info(f"Распакован: {member.name} из {tar_file}")
    except (tarfile.ReadError, EOFError) as e:
        logging.error(f"Ошибка распаковки {tar_file}: {e}")

In [ ]:
# Функция для обработки XML файлов и сохранения информации в CSV
def process_xml_files(directory, output_csv):
    all_data = []  # Список для хранения информации по всем статьям

    # Получаем список всех XML файлов в директории
    xml_files = [f for f in os.listdir(directory) if f.endswith('.nxml') or f.endswith('.xml')]

    for file_name in tqdm(xml_files, desc="Обработка", unit="file", ncols=100):
        file_path = os.path.join(directory, file_name)  # Полный путь к файлу
        with open(file_path, 'r', encoding='utf-8') as file:
            soup = BeautifulSoup(file.read(), 'lxml')  # Парсим XML файл с помощью BeautifulSoup

            # Извлечение названия статьи
            title = soup.find('article-title').get_text(strip=True) if soup.find('article-title') else None

            # Извлечение авторов
            authors = []
            for contrib in soup.find_all('contrib', {'contrib-type': 'author'}):
                surname = contrib.find('surname').get_text(strip=True) if contrib.find('surname') else ''
                given_names = contrib.find('given-names').get_text(strip=True) if contrib.find('given-names') else ''
                authors.append(f"{given_names} {surname}".strip())  # Формируем полное имя автора
            authors = ', '.join(authors)  # Объединяем имена авторов через запятую

            # Извлечение ссылки на статью PMC
            pmc_id = soup.find('article-id', {'pub-id-type': 'pmc'})
            pmc_link = f"https://pmc.ncbi.nlm.nih.gov/articles/PMC{pmc_id.get_text(strip=True)}/" if pmc_id else None

            # Извлечение основного текста статьи
            body = soup.find('body')
            full_text = ''
            if body:
                # Ищем все секции <sec> внутри <body>
                sections = body.find_all('sec')
                for sec in sections:
                    # Объединяем текст всех <p> внутри текущей секции
                    paragraphs = sec.find_all('p', recursive=False)
                    sec_text = ' '.join(para.get_text(strip=True) for para in paragraphs if para.get_text(strip=True))

                    if sec_text:  # Проверяем, что текст не пустой
                        full_text += sec_text + '\n\n'  # Добавляем пробел между абзацами

            # Добавление извлеченной информации в общий список, если текст не пустой
            if title and authors and pmc_link and full_text.strip():
                all_data.append({
                    'Title': title,
                    'Authors': authors,
                    'PMC Link': pmc_link,
                    'Text': full_text.strip()  # Обрезаем лишние пробелы
                })

    # Сохранение всех данных в CSV файл
    pd.DataFrame(all_data).to_csv(output_csv, index=False)
    logging.info(f"Сохранен CSV файл: {output_csv}")

In [ ]:
# Основной цикл обработки каждого base_url
csv_counter = 0
for base_url in base_urls:
    download_all_files(base_url, max_files)

    # Распаковка всех загруженных tar.gz файлов
    tar_files = [f for f in os.listdir(save_directory) if f.endswith('.tar.gz')]
    for tar_file in tqdm(tar_files, desc="Распаковка", unit="file", ncols=100):
        extract_xml_files(os.path.join(save_directory, tar_file))

    # Удаление tar.gz файлов после распаковки
    for tar_file in tar_files:
        try:
            os.remove(os.path.join(save_directory, tar_file))
            logging.info(f"Удалён файл: {tar_file}")
        except OSError as e:
            logging.error(f"Ошибка удаления {tar_file}: {e}")

    # Определение имени выходного CSV файла
    output_csv = f"{csv_output_dir}/cleaned_articles_{csv_counter:02d}.csv"
    process_xml_files(extracted_directory, output_csv)

    # Удаление распакованных XML файлов после обработки
    xml_files = [f for f in os.listdir(extracted_directory) if f.endswith('.nxml') or f.endswith('.xml')]
    for xml_file in xml_files:
        try:
            os.remove(os.path.join(extracted_directory, xml_file))
            logging.info(f"Удалён файл: {xml_file}")
        except OSError as e:
            logging.error(f"Ошибка удаления {xml_file}: {e}")

    csv_counter += 1

Обработка:   8%|███▋                                         | 1524/18673 [02:43<33:12,  8.61file/s]<ipython-input-9-75b1c4b4a697>:11: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(file.read(), 'lxml')  # Парсим XML файл с помощью BeautifulSoup
Обработка: 100%|████████████████████████████████████████████| 18895/18895 [33:50<00:00,  9.30file/s]
